# Лабораторна робота №1. Лінійна регресія.

### Виконала студентка групи КМ-01мн Григоренко Анастасія

In [7]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import linalg 
from scipy.stats import t as ts

In [8]:
pd.set_option('display.max_rows', None)
df=pd.read_csv("MN-03-Hryhorenko.csv", sep = ';', header = None)
df.rename(columns = {0:'x1',1:'x2',2:'x3',3:'x4',4:'x5',5:'x6',6:'x7',7:'y'}, inplace = True)

In [11]:
Y = df[['y']]

In [12]:
df_test = df.iloc[200:250]

In [13]:
Y_train = Y.iloc[0:200]
Y_test = Y.iloc[200:250]

In [14]:
X = df[['x1','x2','x3','x4','x5','x6','x7']]

In [15]:
values=[]
for i in range(250):
    values.append(1)
X.insert(0,'x0', values, allow_duplicates=False)

In [16]:
X_train = X.iloc[0:200]
X_test = X.iloc[200:250]

In [17]:
X1 = df[['x1']]
X1.insert(0,'x0', values, allow_duplicates=False)
X1_train = X1.iloc[0:200]
X1_test = X1.iloc[200:250]

## Метод найменших квадратів

In [19]:
Xt=X_train.transpose()
b = (np.linalg.inv(Xt.dot(X_train))).dot(np.dot(Xt,Y_train))
b

array([[ 3.84887222],
       [-2.84384744],
       [ 9.52540507],
       [ 3.61724474],
       [ 1.38792175],
       [-4.8348413 ],
       [-3.33686675],
       [12.58074713]])

### Визначення коефіцієнту детермінації та перевірка статистичної значимості отриманого рівняння регресії

In [20]:
y_mean=Y_train.mean()

In [21]:
n = 200
Qr = float(np.dot(np.dot(np.transpose(b), Xt), Y_train)) - n*(y_mean**2)
Q = float(np.dot(np.transpose(Y_train), Y_train)) - n*(y_mean**2)

In [22]:
#Коефіцієнт кореляції
R = sqrt(Qr/Q)
R

0.8665464291864959

In [23]:
#Коефіцієнт детермінації
R2 = R**2
R2

0.7509027139358668

In [24]:
#Скоригований коефіцієнт детермінації
R_adj = 1 - ((n-1)/(n-7-1))*(1-R2)
R_adj

0.741821042048112

In [25]:
#Перевірка статистичної значимості
F = (R2*(n-7-1))/((1-R2)*7)
F

82.68331241391091

Порівнюємо отримане значення з табличним значенням F-критерію Фішера при рівні значущості 0.05. Враховуючи степені свободи k1=7 та k2=192, отримуємо Fтеор=2.135, що є меньшим від розрахованого. Отже, побудоване рівняння регресії є значимим, тобто залежна змінна Y достатньо добре описується включеними у регресійну модель змінними x1,x2,x3,x4,x5,x6,x7.

### Обчислення похибки збудованої моделі 

In [26]:
#Похибка
Y_predict = X_test.dot(b)

In [27]:
E2=(Y_test.T.to_numpy()-Y_predict.T.to_numpy())**2
Sigma = sqrt(E2.sum()/42)
Sigma

1.4036405341522913

Похибка отриманої регресійної моделі була обрахована в якості значення σ-середньоквадратичної помилки оцінки, тобто середньоквдратичного відхилення обрахованих даних від фактичних. σ вимірюється в тих самих одиницях виміру, що і Y. Тобто в середньому Y-обраховане відхиляється від Y-фактичне на 1.4 одиниці.


### Поелементні похибки

In [28]:
df_test['MNK'] = np.dot(X1_test,b)
df_test['Delta MNK'] = np.abs(df_test['y'] - df_test['MNK'] )
df_test

ValueError: shapes (50,2) and (8,1) not aligned: 2 (dim 1) != 8 (dim 0)

### Статистична значимість коефіцієнтів

In [29]:
koefs = b.T[0].copy()
Z = np.dot(X_train.transpose(), X_train)
z = linalg.inv(Z).diagonal()

In [30]:
Y_train['MNK'] = np.dot(X_train, koefs) 
s = ((Y_train['MNK'] - Y_train['y'])**2).sum()/(n-7-1)
sb = (s*z)**(1/2)
t = koefs/sb

C:\Users\Nastya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [33]:
t_tabl = 0.3984
for ti, bi in zip(t, b):
    if np.abs(ti) < t_tabl:
        print(f'коеффициент {bi} статистически не значим')
    else:
        print(f'коеффициент {bi} статистически значим')

коеффициент [3.84887222] статистически значим
коеффициент [-2.84384744] статистически значим
коеффициент [9.52540507] статистически значим
коеффициент [3.61724474] статистически значим
коеффициент [1.38792175] статистически значим
коеффициент [-4.8348413] статистически значим
коеффициент [-3.33686675] статистически значим
коеффициент [12.58074713] статистически значим


## Градієнтний спуск

In [34]:
w_current= np.array([0,0,0,0,0,0,0,0])
# w_next = np.array([0,0,0,0,0,0,0,0])

i = 1
while True:
    n = 3/i
    Grad = (2/200)*(np.dot(Xt, np.transpose(np.dot(X_train, w_current)-Y_train.T.to_numpy())))
    w_next = (w_current - n*Grad.T)[0]
    if np.linalg.norm(w_current-w_next) < 0.0001:
        break
    w_current = w_next
    i += 1
#     print(w_current)

In [35]:
w = w_current
w

array([ 4.43651331,  2.08206035,  1.72897503,  0.60390583,  1.93007125,
       -0.10017445,  0.36378047,  1.09937891])

### Обчислення похибки збудованої моделі 

In [36]:
Y_predict_grad = X_test.dot(w)

In [37]:
E2_grad=(Y_test.T.to_numpy()-Y_predict_grad.T.to_numpy())**2
Sigma_grad = sqrt(E2_grad.sum()/42)
Sigma_grad

1.4128970253741735

Похибка отриманої регресійної моделі була обрахована в якості значення σ-середньоквадратичної помилки оцінки, тобто середньоквдратичного відхилення обрахованих даних від фактичних. σ вимірюється в тих самих одиницях виміру, що і Y. Тобто в середньому Y-обраховане відхиляється від Y-фактичне на 1.41 одиницю.

### Поелементні похибки 

In [38]:
df_test['Grad'] = np.dot(X_test,w)
df_test['Delta Grad'] = np.abs(df_test['y'] - df_test['Grad'] )
df_test

C:\Users\Nastya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Nastya\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,x1,x2,x3,x4,x5,x6,x7,y,Grad,Delta Grad
200,0.670,0.510,0.180,1.6800,0.9260,0.2975,0.3935,13,10.512563,2.487437
201,0.675,0.550,0.190,1.5510,0.7105,0.3685,0.4120,13,10.416946,2.583054
202,0.680,0.520,0.165,1.4775,0.7240,0.2790,0.4060,11,10.178022,0.821978
203,0.680,0.530,0.180,1.5290,0.7635,0.3115,0.4025,11,10.307788,0.692212
204,0.700,0.525,0.175,1.7585,0.8745,0.3615,0.4700,10,10.861993,0.861993
205,0.700,0.550,0.200,1.5230,0.6930,0.3060,0.4405,13,10.431344,2.568656
206,0.725,0.530,0.190,1.7315,0.8300,0.3980,0.4050,11,10.825913,0.174087
207,0.725,0.550,0.200,1.5100,0.8735,0.4265,0.5085,9,10.558816,1.558816
208,0.735,0.570,0.175,1.8800,0.9095,0.3870,0.4880,11,11.272732,0.272732
209,0.740,0.575,0.220,2.0120,0.8915,0.5265,0.4710,12,11.607594,0.392406
